# Cython
---
- Making python code faster, particularly things that can't be done in scipy/numpy
- Wrapping/interfacing with C/C++ code

### Cython install
---

```bash
pip3 install cython
```
---

### Why Cyhton
---
In Python, when you declare a variable, like:

`x = 5.0`
You never had to tell the language that the variable 'x' was an integer. In fact, later, you can assign 'Gary' to x and be just fine. This is because Python checks every single time for you to figure out the type. This is called "dynamic typing."

This is nifty, and makes learning initially very simple, and Python was really only meant to be a teaching language, but this severely slows things down.

Instead, if we're willing to, we can use static typing and Cython to get some serious speed ups. Many languages do something more like:
`float x = 5.0` <br>

Cython wants something like:
`cdef float x = 5.0`

### Function in python 

In [7]:
#example_original.py
def test(x):
    y = 0
    for i in range(x):
        y += i
    return y

- How do we prepare this file to be passed through Cython? rather than .py, we save file with .pyx extension

In [3]:
#example_cython.pyx
def test(x):
    y = 0
    for i in range(x):
        y += i
    return y

- Once you have a .pyx, you're ready to build. To do this, we're going to make a **setup.py** file

#### below code must not be run in jupyter notebook it should be save in file and then run with in terminal with commands given below

In [ ]:
# setup.py
from distutils.core import setup
from Cython.Build import cythonize

setup(ext_modules = cythonize('example_cython.pyx'))

**Now run the below command in terminal**

In [4]:
!python3 setup.py build_ext --inplace

Compiling example_cython.pyx because it changed.
[1/1] Cythonizing example_cython.pyx
/home/vaibhav/.local/lib/python3.8/site-packages/Cython/Compiler/Main.py:369: FutureWarning: Cython directive 'language_level' not set, using 2 for now (Py2). This will change in a later release! File: /home/vaibhav/Desktop/projects/freelance/documentation/example_cython.pyx
  tree = Parsing.p_module(s, pxd, full_module_name)
running build_ext
building 'example_cython' extension
creating build
creating build/temp.linux-x86_64-3.8
x86_64-linux-gnu-gcc -pthread -Wno-unused-result -Wsign-compare -DNDEBUG -g -fwrapv -O2 -Wall -g -fstack-protector-strong -Wformat -Werror=format-security -g -fwrapv -O2 -g -fstack-protector-strong -Wformat -Werror=format-security -Wdate-time -D_FORTIFY_SOURCE=2 -fPIC -I/usr/include/python3.8 -c example_cython.c -o build/temp.linux-x86_64-3.8/example_cython.o
x86_64-linux-gnu-gcc -pthread -shared -Wl,-O1 -Wl,-Bsymbolic-functions -Wl,-Bsymbolic-functions -Wl,-z,relro -g -fwrap

##### `example_cython.cpython-38-x86_64-linux-gnu.so` 
if you see this in end of terminal then cython has worked properly

--- 
This should create a `build directory, a C file (.c), and a Shared Object file (.so)`. With this, we can import our C-extension


### Using the shared object (.so file)
- create a new file `testing.py` which will use the function we defined in cython and we will use it here.

In [5]:
#testing.py
import example_cython

example_cython.test(5)

10

In [8]:
test(5)

10

#### This is how we imported a cython function which we defined earlier 
---

### More on using Cython to write optimized code
---

#### cdef declarations
- cdef int x,y,z
- cdef char *s
- cdef float x = 5.2 (single precision)
- cdef double x = 40.5 (double precision)
- cdef list languages
- cdef dict abc_dict
- cdef object thing

#### def, cdef, and cpdef
- **def** - regular python function, calls from Python only.
- **cdef** - cython only functions, can't access these from python-only code, must access within Cython, since there will be no C translation to Python for these.
- **cpdef** - C and Python. Will create a C function and a wrapper for Python. Why not *always* use cpdef? In some cases, you might have C only pointer, like a C array. We'll be mostly using cpdef, however.

Starting again with orignal example

In [27]:
#example_original.py
def test(x):
    y = 0
    for i in range(x):
        y += i
    return y

Changes in .pyx file  || again can not run this function here in jupyter need to save it in file as .pyx

In [ ]:
#example_cython.pyx
def test(int x):
    cdef int y = 0
    cdef int i
    for i in range(x):
        y += i
    return y

Code in setup.py file to make the .so object which we can use anywhere || again can not run this function here in jupyter need to save it in file as .py and run with the below commands

In [ ]:
#setup.py
from distutils.core import setup
from Cython.Build import cythonize

setup(ext_modules = cythonize('example_cython.pyx'))

**Code for terminal to run the setup.py file**

In [22]:
!python3 setup.py build_ext --inplace

running build_ext


#### Testing the execution time of cython function vs python function 

In [39]:
%timeit test(500)

19.4 µs ± 1.89 µs per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [40]:
import example_cython

In [41]:
%timeit example_cython.test(500)

11.6 µs ± 32.5 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


### For the same function writen in cython it is 2x faster then python